In [2]:
!pip install simpledbf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for simpledbf: filename=simpledbf-0.2.6-py3-none-any.whl size=13785 sha256=2f62535fdcfc99a223079e779567b291f3e580acabfa60f85b1eb1bb6e316f7e
  Stored in directory: /root/.cache/pip/wheels/e5/41/13/ebdef29165b9309ec4e235dbff19eca8b6759125b0924ad430
Successfully built simpledbf


In [3]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
import re, os
from pathlib import Path
from glob import glob
from simpledbf import Dbf5
import matplotlib.pyplot as plt
from statistics import mode

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import Ridge
from sklearn.metrics import log_loss

def Read():
  train = pd.read_csv(path_start/"input/df_train.csv")
  test = pd.read_csv(path_start/"input/test.csv")
  df_ConCat = pd.concat([train, test])
  df_ConCat = df_ConCat.drop(["Unnamed: 0", "地域", "前面道路：方位", "前面道路：種類",	"前面道路：幅員（ｍ）", 
                              "種類", "土地の形状", "間口", "延床面積（㎡）"], axis=1)
  return df_ConCat

path_start = Path("/content/drive/MyDrive/Nishika/Mansion_2023_spring")
path_import = path_start/"Inport_data/S12-21_GML"

Mounted at /content/drive


In [4]:
df_station = Dbf5(path_import/'utf8/S12-21_NumberOfPassengers.dbf', codec='utf8')
df = Read()
df_station = df_station.to_dataframe()
df_station = df_station.drop_duplicates(subset='S12_001')
print(df_station.shape)
display(df_station.head())

(8704, 45)


,S12_001,S12_002,S12_003,S12_004,S12_005,S12_006,S12_007,S12_008,S12_009,S12_010,...,S12_036,S12_037,S12_038,S12_039,S12_040,S12_041,S12_042,S12_043,S12_044,S12_045
0,二月田,九州旅客鉄道,指宿枕崎線,11,2,1,3,NaN,0,1,...,NaN,658,1,1,NaN,690,1,1,NaN,318
1,古島,沖縄都市モノレール,沖縄都市モノレール線,23,5,1,1,NaN,3907,1,...,NaN,5086,1,1,NaN,5143,1,1,NaN,3404
2,お台場海浜公園,ゆりかもめ,東京臨海新交通臨海線,24,5,1,1,NaN,14612,1,...,NaN,17165,1,1,NaN,16899,1,1,NaN,7703
3,東京国際クルーズターミナル,ゆりかもめ,東京臨海新交通臨海線,24,5,1,1,NaN,3767,1,...,NaN,3551,1,1,NaN,3191,1,1,NaN,1383
4,テレコムセンター,ゆりかもめ,東京臨海新交通臨海線,24,5,1,1,NaN,12112,1,...,NaN,13475,1,1,NaN,12140,1,1,NaN,6536


S12_008 = 備考2011, 
S12_012 = 備考2012, 
S12_016 = 備考2013, 
S12_020 = 備考2014, 
S12_024 = 備考2015, 
S12_028 = 備考2016, 
S12_032 = 備考2017, 
S12_036 = 備考2018, 
S12_040 = 備考2019, 
S12_044 = 備考2020

In [5]:
my_dict = {}
List_test = ["S12_{}".format(str(i).zfill(3)) for i in range(9,46) if i%4==1]
List_year = ["駅別乗降客数_{}".format(i+2011) for i in range(len(List_test))]

for i in range(len(List_test)):
    key = List_test[i]
    value = List_year[i]
    my_dict[key] = value
    
a = {"S12_001":"最寄駅：名称", "S12_002":"運営会社", "S12_003":"路線名"}
a.update(my_dict)
df_stage1 = df_station[["S12_001", "S12_002", "S12_003"] + List_test].rename(columns=a)
df_stage1

,最寄駅：名称,運営会社,路線名,駅別乗降客数_2011,駅別乗降客数_2012,駅別乗降客数_2013,駅別乗降客数_2014,駅別乗降客数_2015,駅別乗降客数_2016,駅別乗降客数_2017,駅別乗降客数_2018,駅別乗降客数_2019,駅別乗降客数_2020
0,二月田,九州旅客鉄道,指宿枕崎線,0,0,0,0,0,0,0,658,690,318
1,古島,沖縄都市モノレール,沖縄都市モノレール線,3907,3980,4572,4187,4648,4528,4819,5086,5143,3404
2,お台場海浜公園,ゆりかもめ,東京臨海新交通臨海線,14612,16130,16357,16515,17537,17488,16944,17165,16899,7703
3,東京国際クルーズターミナル,ゆりかもめ,東京臨海新交通臨海線,3767,3235,3407,3782,3682,3682,3590,3551,3191,1383
4,テレコムセンター,ゆりかもめ,東京臨海新交通臨海線,12112,12775,13366,14138,14069,14069,13744,13475,12140,6536
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10466,美乃浜学園,ひたちなか海浜鉄道,湊線,0,0,0,0,0,0,0,0,0,10
10467,えちご押上ひすい海岸,えちごトキめき鉄道,日本海ひすいライン,0,0,0,0,0,0,0,0,0,128
10468,オークスカナルパークホテル富山前,富山地方鉄道,富山港線,0,0,0,0,0,0,0,0,0,0
10469,龍谷富山高校前（永楽町）,富山地方鉄道,富山港線,0,0,0,0,0,0,0,0,0,0


In [6]:
def MARUKOFU_MODOKI(df_stage1, x):
  a = df_stage1.iloc[x, :]
  pred = 0
  i = 1
  weight = 1.5

  while str(type(a[-i]))!="<class 'str'>":
    pred += a[-i]*np.exp(-i/weight)
    i+=1
  return pred

MARUKOFU_MODOKI(df_stage1, 0)

434.1992855219069

In [7]:
year = 2021
for h in range(2):
  keep = []
  for i in range(len(df_stage1)):
    keep.append(int(round(MARUKOFU_MODOKI(df_stage1, i))))
  df_stage1["駅別乗降客数_{}".format(year+h)] = keep
df_stage1

,最寄駅：名称,運営会社,路線名,駅別乗降客数_2011,駅別乗降客数_2012,駅別乗降客数_2013,駅別乗降客数_2014,駅別乗降客数_2015,駅別乗降客数_2016,駅別乗降客数_2017,駅別乗降客数_2018,駅別乗降客数_2019,駅別乗降客数_2020,駅別乗降客数_2021,駅別乗降客数_2022
0,二月田,九州旅客鉄道,指宿枕崎線,0,0,0,0,0,0,0,658,690,318,434,446
1,古島,沖縄都市モノレール,沖縄都市モノレール線,3907,3980,4572,4187,4648,4528,4819,5086,5143,3404,4449,4569
2,お台場海浜公園,ゆりかもめ,東京臨海新交通臨海線,14612,16130,16357,16515,17537,17488,16944,17165,16899,7703,13148,13501
3,東京国際クルーズターミナル,ゆりかもめ,東京臨海新交通臨海線,3767,3235,3407,3782,3682,3682,3590,3551,3191,1383,2545,2613
4,テレコムセンター,ゆりかもめ,東京臨海新交通臨海線,12112,12775,13366,14138,14069,14069,13744,13475,12140,6536,10339,10616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10466,美乃浜学園,ひたちなか海浜鉄道,湊線,0,0,0,0,0,0,0,0,0,10,5,5
10467,えちご押上ひすい海岸,えちごトキめき鉄道,日本海ひすいライン,0,0,0,0,0,0,0,0,0,128,66,68
10468,オークスカナルパークホテル富山前,富山地方鉄道,富山港線,0,0,0,0,0,0,0,0,0,0,0,0
10469,龍谷富山高校前（永楽町）,富山地方鉄道,富山港線,0,0,0,0,0,0,0,0,0,0,0,0


In [8]:
df_a = pd.merge(df, df_stage1, on="最寄駅：名称", how="left")
df_b = df_a[["ID"] + list(df_stage1.columns)]

filling = {"最寄駅：名称":"不明", "運営会社":"不明", "路線名":"不明"}
df_c = df_b.fillna(filling)
df_c = df_c.fillna(-1)
df_c.head(-5)

,ID,最寄駅：名称,運営会社,路線名,駅別乗降客数_2011,駅別乗降客数_2012,駅別乗降客数_2013,駅別乗降客数_2014,駅別乗降客数_2015,駅別乗降客数_2016,駅別乗降客数_2017,駅別乗降客数_2018,駅別乗降客数_2019,駅別乗降客数_2020,駅別乗降客数_2021,駅別乗降客数_2022
0,8067540,稲戸井,関東鉄道,常総線,1661.0,1712.0,1670.0,1610.0,1514.0,1478.0,1488.0,1442.0,1458.0,1181.0,1399.0,1437.0
1,8027265,ひたち野うしく,東日本旅客鉄道,常磐線,11630.0,11698.0,12224.0,12878.0,12632.0,13314.0,13574.0,13946.0,13946.0,9914.0,12527.0,12863.0
2,8061526,研究学園,首都圏新都市鉄道,常磐新線,9800.0,10600.0,11900.0,12200.0,13000.0,13600.0,14100.0,14600.0,14600.0,9600.0,12667.0,13007.0
3,8086147,つくば,首都圏新都市鉄道,常磐新線,31200.0,33300.0,34800.0,34300.0,35700.0,36900.0,35800.0,36400.0,36100.0,21000.0,30294.0,31107.0
4,8049498,守谷,関東鉄道,常総線,11361.0,12275.0,12889.0,13922.0,13973.0,15151.0,15630.0,14347.0,14638.0,10500.0,13312.0,13669.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
784703,47001695,不明,不明,不明,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
784704,47001723,古島,沖縄都市モノレール,沖縄都市モノレール線,3907.0,3980.0,4572.0,4187.0,4648.0,4528.0,4819.0,5086.0,5143.0,3404.0,4449.0,4569.0
784705,47001780,安里,沖縄都市モノレール,沖縄都市モノレール線,3275.0,3302.0,3836.0,3593.0,3910.0,3941.0,4149.0,4480.0,4684.0,2853.0,3871.0,3975.0
784706,47003330,不明,不明,不明,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [16]:
scaler = StandardScaler()
scaler.fit(df_c[list(df_c.columns[4:])])
df_c[list(df_c.columns[4:])] = scaler.transform(df_c[list(df_c.columns[4:])])
df_c.head()

,ID,最寄駅：名称,運営会社,路線名,駅別乗降客数_2011,駅別乗降客数_2012,駅別乗降客数_2013,駅別乗降客数_2014,駅別乗降客数_2015,駅別乗降客数_2016,駅別乗降客数_2017,駅別乗降客数_2018,駅別乗降客数_2019,駅別乗降客数_2020,駅別乗降客数_2021,駅別乗降客数_2022
0,8067540,稲戸井,関東鉄道,常総線,-0.488452,-0.569343,-0.575882,-0.599380,-0.608130,-0.615370,-0.614987,-0.604885,-0.616000,-0.639725,-0.630697,-0.630688
1,8027265,ひたち野うしく,東日本旅客鉄道,常磐線,-0.332201,-0.390694,-0.389167,-0.393209,-0.410235,-0.405988,-0.404771,-0.403159,-0.407023,-0.426199,-0.418776,-0.418779
2,8061526,研究学園,首都圏新都市鉄道,常磐新線,-0.360884,-0.410338,-0.394899,-0.405614,-0.403684,-0.400928,-0.395623,-0.392609,-0.396078,-0.433877,-0.416110,-0.416108
3,8086147,つくば,首都圏新都市鉄道,常磐新線,-0.025468,-0.004236,0.010233,-0.001249,0.000366,0.011256,-0.018188,-0.040911,-0.036291,-0.155142,-0.080423,-0.080421
4,8049498,守谷,関東鉄道,常総線,-0.336418,-0.380372,-0.377403,-0.374107,-0.386366,-0.373490,-0.369011,-0.396690,-0.395442,-0.411871,-0.403826,-0.403831


## 出力

In [17]:
df_c.to_csv(path_start/"feature_value/df_駅別乗降客数.csv", index=False)
pd.read_csv(path_start/"feature_value/df_駅別乗降客数.csv")

,ID,最寄駅：名称,運営会社,路線名,駅別乗降客数_2011,駅別乗降客数_2012,駅別乗降客数_2013,駅別乗降客数_2014,駅別乗降客数_2015,駅別乗降客数_2016,駅別乗降客数_2017,駅別乗降客数_2018,駅別乗降客数_2019,駅別乗降客数_2020,駅別乗降客数_2021,駅別乗降客数_2022
0,8067540,稲戸井,関東鉄道,常総線,-0.488452,-0.569343,-0.575882,-0.599380,-0.608130,-0.615370,-0.614987,-0.604885,-0.616000,-0.639725,-0.630697,-0.630688
1,8027265,ひたち野うしく,東日本旅客鉄道,常磐線,-0.332201,-0.390694,-0.389167,-0.393209,-0.410235,-0.405988,-0.404771,-0.403159,-0.407023,-0.426199,-0.418776,-0.418779
2,8061526,研究学園,首都圏新都市鉄道,常磐新線,-0.360884,-0.410338,-0.394899,-0.405614,-0.403684,-0.400928,-0.395623,-0.392609,-0.396078,-0.433877,-0.416110,-0.416108
3,8086147,つくば,首都圏新都市鉄道,常磐新線,-0.025468,-0.004236,0.010233,-0.001249,0.000366,0.011256,-0.018188,-0.040911,-0.036291,-0.155142,-0.080423,-0.080421
4,8049498,守谷,関東鉄道,常総線,-0.336418,-0.380372,-0.377403,-0.374107,-0.386366,-0.373490,-0.369011,-0.396690,-0.395442,-0.411871,-0.403826,-0.403831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
784708,47006020,不明,不明,不明,-0.514502,-0.599989,-0.605444,-0.628856,-0.635097,-0.641534,-0.640885,-0.628165,-0.640416,-0.668625,-0.657358,-0.657358
784709,47006229,不明,不明,不明,-0.514502,-0.599989,-0.605444,-0.628856,-0.635097,-0.641534,-0.640885,-0.628165,-0.640416,-0.668625,-0.657358,-0.657358
784710,47006331,不明,不明,不明,-0.514502,-0.599989,-0.605444,-0.628856,-0.635097,-0.641534,-0.640885,-0.628165,-0.640416,-0.668625,-0.657358,-0.657358
784711,47006332,不明,不明,不明,-0.514502,-0.599989,-0.605444,-0.628856,-0.635097,-0.641534,-0.640885,-0.628165,-0.640416,-0.668625,-0.657358,-0.657358
